### Imports

In [30]:
import pandas as pd

In [31]:
df = pd.read_csv("./data/frmgham2_dirty_modified.csv")
df.head()

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0,39,106.0,70.0,No,0.0,26.97,No,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.0,52,121.0,66.0,No,0.0,NaN,No,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.0,46,121.0,81.0,No,0.0,28.73,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.0,52,105.0,69.5,No,0.0,29.43,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.0,58,108.0,66.0,No,0.0,28.50,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766


In [32]:
df_50 = pd.read_csv("./data/frmgham2_sample_50_a.csv")
df_50.head()

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,7055406,2,300.0,60,118.0,72.0,Yes,20.0,27.32,No,...,1,0,7624,8766,8766,7624,8766,7624,8766,8766
1,1969205,1,212.0,64,142.0,80.0,No,0.0,24.07,No,...,0,1,8766,8766,8766,8766,8766,8766,8766,5235
2,473381,1,206.0,54,150.0,92.0,Yes,20.0,25.34,No,...,0,1,8766,8766,8766,8766,8766,8766,8766,4328
3,2928737,2,214.0,47,131.0,78.0,No,0.0,26.51,No,...,0,1,8766,8766,8766,8766,8766,8766,8766,6617
4,9027228,1,262.0,53,104.0,69.0,No,0.0,27.52,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766


### Modification

Make a clean copy.

In [33]:
df_no_missing = df.copy()
df_no_missingn = df_no_missing.dropna()
df_no_missing = df_no_missing[(df_no_missing["educ"] != 0) & (df_no_missing["AGE"] != -1)]
df_no_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11224 entries, 0 to 11631
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   RANDID    11224 non-null  int64  
 1   SEX       11224 non-null  object 
 2   TOTCHOL   10834 non-null  float64
 3   AGE       11224 non-null  int64  
 4   SYSBP     11224 non-null  float64
 5   DIABP     11224 non-null  float64
 6   CURSMOKE  11224 non-null  object 
 7   CIGPDAY   11151 non-null  float64
 8   BMI       11174 non-null  float64
 9   DIABETES  11224 non-null  object 
 10  BPMEDS    10644 non-null  float64
 11  HEARTRTE  11218 non-null  float64
 12  GLUCOSE   9822 non-null   float64
 13  educ      11224 non-null  float64
 14  PREVCHD   11224 non-null  int64  
 15  PREVAP    11224 non-null  int64  
 16  PREVMI    11224 non-null  int64  
 17  PREVSTRK  11224 non-null  int64  
 18  PREVHYP   11224 non-null  int64  
 19  TIME      11224 non-null  int64  
 20  PERIOD    11224 non-null  in

#### Preparation For Association Rules

In [34]:
df_ar = df_no_missing.copy()
df_ar = df_ar[["RANDID", "PERIOD", "DIABETES", "STROKE", "HYPERTEN", "ANYCHD"]]
df_ar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11224 entries, 0 to 11631
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   RANDID    11224 non-null  int64 
 1   PERIOD    11224 non-null  int64 
 2   DIABETES  11224 non-null  object
 3   STROKE    11224 non-null  int64 
 4   HYPERTEN  11224 non-null  int64 
 5   ANYCHD    11224 non-null  object
dtypes: int64(4), object(2)
memory usage: 613.8+ KB


In [35]:
df_ar = df_ar[df_ar["PERIOD"] == 3]
df_ar = df_ar.drop("PERIOD", axis = 1)
df_ar.head()

,RANDID,DIABETES,STROKE,HYPERTEN,ANYCHD
1,2448,No,0,0,Yes
4,6238,No,0,0,No
11,11252,No,0,1,No
14,11263,Yes,0,1,Yes
19,12806,No,0,1,No


In [36]:
def create_transactions(start_index):
  df_ar_trans = pd.DataFrame()

  for index, data in df_ar.iterrows():
    if data["DIABETES"] == "Yes":
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["DIABETES"] })])
    if data["STROKE"] == 1:
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["STROKE"] })])
    if data["HYPERTEN"] == 1:
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["HYPERTEN"] })])
    if data["ANYCHD"] == "Yes":
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["ANYCHD"] })])

  df_ar_trans = df_ar_trans.reset_index(drop=True)

  df_ar_trans = df_ar_trans[start_index:start_index + 50]

  print(df_ar_trans)

  return df_ar_trans

In [37]:
df_ar_trans_sample_a = create_transactions(2)

    RANDID Condition
2    11263  DIABETES
3    11263  HYPERTEN
4    11263    ANYCHD
5    12806  HYPERTEN
6    14367  HYPERTEN
7    16365  HYPERTEN
8    23727  HYPERTEN
9    24721  HYPERTEN
10   33077  HYPERTEN
11   34689  HYPERTEN
12   36459  HYPERTEN
13   40435  HYPERTEN
14   43522  HYPERTEN
15   43770  DIABETES
16   43770  HYPERTEN
17   43770    ANYCHD
18   45464  HYPERTEN
19   47561  HYPERTEN
20   55965  HYPERTEN
21   55965    ANYCHD
22   63156  HYPERTEN
23   66472  DIABETES
24   66472    STROKE
25   66472  HYPERTEN
26   67905  HYPERTEN
27   68194  HYPERTEN
28   68397  HYPERTEN
29   68397    ANYCHD
30   77492  HYPERTEN
31   77492    ANYCHD
32   82188  HYPERTEN
33   83398  DIABETES
34   83398    STROKE
35   83398  HYPERTEN
36   84815    STROKE
37   84815  HYPERTEN
38   95148  HYPERTEN
39   95148    ANYCHD
40   95182  HYPERTEN
41   95182    ANYCHD
42   97895    STROKE
43   97895  HYPERTEN
44   97895    ANYCHD
45   98536  HYPERTEN
46  101990  HYPERTEN
47  101990    ANYCHD
48  112297  H

In [38]:
df_ar_trans_sample_b = create_transactions(5)

    RANDID Condition
5    12806  HYPERTEN
6    14367  HYPERTEN
7    16365  HYPERTEN
8    23727  HYPERTEN
9    24721  HYPERTEN
10   33077  HYPERTEN
11   34689  HYPERTEN
12   36459  HYPERTEN
13   40435  HYPERTEN
14   43522  HYPERTEN
15   43770  DIABETES
16   43770  HYPERTEN
17   43770    ANYCHD
18   45464  HYPERTEN
19   47561  HYPERTEN
20   55965  HYPERTEN
21   55965    ANYCHD
22   63156  HYPERTEN
23   66472  DIABETES
24   66472    STROKE
25   66472  HYPERTEN
26   67905  HYPERTEN
27   68194  HYPERTEN
28   68397  HYPERTEN
29   68397    ANYCHD
30   77492  HYPERTEN
31   77492    ANYCHD
32   82188  HYPERTEN
33   83398  DIABETES
34   83398    STROKE
35   83398  HYPERTEN
36   84815    STROKE
37   84815  HYPERTEN
38   95148  HYPERTEN
39   95148    ANYCHD
40   95182  HYPERTEN
41   95182    ANYCHD
42   97895    STROKE
43   97895  HYPERTEN
44   97895    ANYCHD
45   98536  HYPERTEN
46  101990  HYPERTEN
47  101990    ANYCHD
48  112297  HYPERTEN
49  112469  HYPERTEN
50  123622  HYPERTEN
51  125413  H

In [39]:
df_ar_trans_sample_c = create_transactions(6)

    RANDID Condition
6    14367  HYPERTEN
7    16365  HYPERTEN
8    23727  HYPERTEN
9    24721  HYPERTEN
10   33077  HYPERTEN
11   34689  HYPERTEN
12   36459  HYPERTEN
13   40435  HYPERTEN
14   43522  HYPERTEN
15   43770  DIABETES
16   43770  HYPERTEN
17   43770    ANYCHD
18   45464  HYPERTEN
19   47561  HYPERTEN
20   55965  HYPERTEN
21   55965    ANYCHD
22   63156  HYPERTEN
23   66472  DIABETES
24   66472    STROKE
25   66472  HYPERTEN
26   67905  HYPERTEN
27   68194  HYPERTEN
28   68397  HYPERTEN
29   68397    ANYCHD
30   77492  HYPERTEN
31   77492    ANYCHD
32   82188  HYPERTEN
33   83398  DIABETES
34   83398    STROKE
35   83398  HYPERTEN
36   84815    STROKE
37   84815  HYPERTEN
38   95148  HYPERTEN
39   95148    ANYCHD
40   95182  HYPERTEN
41   95182    ANYCHD
42   97895    STROKE
43   97895  HYPERTEN
44   97895    ANYCHD
45   98536  HYPERTEN
46  101990  HYPERTEN
47  101990    ANYCHD
48  112297  HYPERTEN
49  112469  HYPERTEN
50  123622  HYPERTEN
51  125413  HYPERTEN
52  127109  H

In [40]:
df_ar_trans_sample_d = create_transactions(7)

    RANDID Condition
7    16365  HYPERTEN
8    23727  HYPERTEN
9    24721  HYPERTEN
10   33077  HYPERTEN
11   34689  HYPERTEN
12   36459  HYPERTEN
13   40435  HYPERTEN
14   43522  HYPERTEN
15   43770  DIABETES
16   43770  HYPERTEN
17   43770    ANYCHD
18   45464  HYPERTEN
19   47561  HYPERTEN
20   55965  HYPERTEN
21   55965    ANYCHD
22   63156  HYPERTEN
23   66472  DIABETES
24   66472    STROKE
25   66472  HYPERTEN
26   67905  HYPERTEN
27   68194  HYPERTEN
28   68397  HYPERTEN
29   68397    ANYCHD
30   77492  HYPERTEN
31   77492    ANYCHD
32   82188  HYPERTEN
33   83398  DIABETES
34   83398    STROKE
35   83398  HYPERTEN
36   84815    STROKE
37   84815  HYPERTEN
38   95148  HYPERTEN
39   95148    ANYCHD
40   95182  HYPERTEN
41   95182    ANYCHD
42   97895    STROKE
43   97895  HYPERTEN
44   97895    ANYCHD
45   98536  HYPERTEN
46  101990  HYPERTEN
47  101990    ANYCHD
48  112297  HYPERTEN
49  112469  HYPERTEN
50  123622  HYPERTEN
51  125413  HYPERTEN
52  127109  HYPERTEN
53  127196   

### Exports

In [41]:
df_ar_trans_sample_a.to_csv("./data/frmgham2_ar_sample_a.csv", index=False)

In [42]:
df_ar_trans_sample_b.to_csv("./data/frmgham2_ar_sample_b.csv", index=False)

In [43]:
df_ar_trans_sample_c.to_csv("./data/frmgham2_ar_sample_c.csv", index=False)

In [44]:
df_ar_trans_sample_d.to_csv("./data/frmgham2_ar_sample_d.csv", index=False)